# from ML model in OCI Artifact → to ModelCar

ModelCar's Dockerfile:

```dockerfile
FROM ghcr.io/oras-project/oras:v1.2.0 as builder

RUN oras pull quay.io/mmortari/ml-iris:v1 
 

FROM busybox

RUN mkdir /models && chmod 775 /models
COPY --from=builder /workspace /models/
```

In [3]:
!podman build --load -t mmortari/ml-iris:v1-modelcar -f Containerfile.modelcar .
# !podman push --tls-verify=false mmortari/ml-iris:v1-modelcar localhost:8080/matteo/ml-iris:v1-modelcar
!podman push mmortari/ml-iris:v1-modelcar quay.io/mmortari/ml-iris:v1-modelcar

[1/2] STEP 1/2: FROM ghcr.io/oras-project/oras:v1.2.0 AS builder
[1/2] STEP 2/2: RUN oras pull quay.io/mmortari/ml-iris:v1 
--> Using cache 7feb1e5fb58481657bd017001bd1f8ce7f930041f522c29ffcee44bc346bf99c
--> 7feb1e5fb584
[2/2] STEP 1/3: FROM busybox
[2/2] STEP 2/3: RUN mkdir /models && chmod 775 /models
--> Using cache 4c41b98df27a711498d7e585a7e6a13cc660dc86dc2a30f45fd4d869e5b65091
--> 4c41b98df27a
[2/2] STEP 3/3: COPY --from=builder /workspace /models/
--> Using cache b6a5b03fd625e3a49fd6bd104d250c6efe1be53f48f23db54a1714513e9eb954
[2/2] COMMIT mmortari/ml-iris:v1-modelcar
--> b6a5b03fd625
Successfully tagged localhost/mmortari/ml-iris:v1-modelcar
Successfully tagged localhost/matteo/ml-iris:v1-modelcar
b6a5b03fd625e3a49fd6bd104d250c6efe1be53f48f23db54a1714513e9eb954
Getting image source signatures
Copying blob sha256:e5744b46b6c629c1861eb438aca266a1a170a519f080db5885cc4e672cd78d1b
Copying blob sha256:8e13bc96641a6983e79c9569873afe5800b0efb3993c3302763b9f5bc5fb8739
Copying blob sha2

| local Quay | Quay.io |
| --- | --- |
| ![image.png](./imgs/Screenshot%202024-06-17%20at%2014.05.02.png) | ![](./imgs/Screenshot%202024-06-24%20at%2013.12.25.png) |

# from ModelCar → to BootC image (linux+server+model[/car])

bootc containerfile (snippet):

```Dockerfile
FROM quay.io/centos-bootc/centos-bootc:stream9
# ...

# Add quadlet files to setup system to automatically run AI application on boot
COPY quadlet/sklearn.kube quadlet/sklearn.yaml /usr/share/containers/systemd/

# Prepull the model, model_server & application images to populate the system.
# Comment the pull commands to keep bootc image smaller.
# The quadlet .image file added above pulls following images with service startup
RUN podman pull --root /usr/lib/containers/storage docker.io/kserve/sklearnserver:latest
RUN podman pull --root /usr/lib/containers/storage quay.io/mmortari/ml-iris:v1-modelcar

# ...
```

In [5]:
!podman build --build-arg "SSHPUBKEY=$(cat ~/.ssh/id_rsa.pub)" \
       --security-opt label=disable \
	   --cap-add SYS_ADMIN \
	   -f Containerfile.bootc \
	   -t mmortari/ml-iris:v1-bootc .
!podman push mmortari/ml-iris:v1-bootc quay.io/mmortari/ml-iris:v1-bootc

STEP 1/9: FROM quay.io/centos-bootc/centos-bootc:stream9
STEP 2/9: ARG SSHPUBKEY
--> Using cache 523580821612112581608763e3943eb40817089f87b690dac045459c0b14fb99
--> 523580821612
STEP 3/9: RUN set -eu; mkdir -p /usr/ssh &&     echo 'AuthorizedKeysFile /usr/ssh/%u.keys .ssh/authorized_keys .ssh/authorized_keys2' >> /etc/ssh/sshd_config.d/30-auth-system.conf &&     echo ${SSHPUBKEY} > /usr/ssh/root.keys && chmod 0600 /usr/ssh/root.keys
--> Using cache 3359a78489d3e4eca5921449532819c1b234660be8ac46f3752dad6ee8989eff
--> 3359a78489d3
STEP 4/9: COPY quadlet/sklearn.kube quadlet/sklearn.yaml /usr/share/containers/systemd/
--> Using cache 5dbe59af0d46b95e74577ce99172e11917622f847d00e4b231cb3d10a937d74a
--> 5dbe59af0d46
STEP 5/9: RUN sed -i -e '/additionalimage.*/a "/usr/lib/containers/storage",'         /etc/containers/storage.conf
--> Using cache e16046b72ce01887444619469f31aa4d758cb7dc8b07c51dd7848cc452349df9
--> e16046b72ce0
STEP 6/9: VOLUME /var/lib/containers
--> Using cache 8c0ce999a83d

Now the bootc container image is available:

![image](./imgs/Screenshot%202024-06-24%20at%2013.22.15.png)

We could also make a Virtual Machine out of it:

![image](./imgs/Screenshot%202024-06-24%20at%2013.26.50.png)

I could run the Virtual Machine and it would serve my model:

![image](./imgs/Screenshot%202024-06-18%20at%2010.35.31_2.png)

and I could interact with it to make Inference:

![](./imgs/Screenshot%202024-06-19%20at%2011.48.25.png)